In [284]:
import pandas as pd
import numpy as np
import sys
from collections import deque

In [285]:
driverInfo = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/driverInfo.csv')
constructorInfo = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/constructorInfo.csv')
quali_results_I = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/qualifying_results_I.csv')
quali_results_II = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/qualifying_results_II.csv')
raceResults = pd.read_csv('/Users/parthu/MyStuff/projects/Formula_One/Data Collection/raceResults.csv')

In [286]:
raceDf = raceResults[['season','round','driverId','constructorId']]
unionDf = pd.merge(raceDf,raceDf,how="left",on=['season','round','constructorId'])
driverMatedf = unionDf.loc[unionDf['driverId_x'] != unionDf['driverId_y']].reset_index(drop=True)

In [287]:
driverMateAgg = driverMatedf.groupby(['constructorId','driverId_x','driverId_y'])['round'].count().reset_index()
driverMateAgg = driverMateAgg.loc[driverMateAgg['round'] > 3]
drivers = driverMateAgg.groupby(['driverId_x'])['driverId_y'].apply(list).reset_index(name='teammates').rename(columns={'driverId_x':'driverID'})

In [288]:
number_drivers = drivers.shape[0]
driver_network = np.zeros((number_drivers,number_drivers),dtype=int)
for i in range(0,number_drivers):
    for j in drivers['teammates'][i]:
        ind = drivers.loc[drivers['driverID'] == j].index
        driver_network[i][ind] = 1

In [289]:
def shortest_path(adj_matrix, start, end):
    num_nodes = adj_matrix.shape[0]
    
    # Initialize distance array
    distance = np.full(num_nodes, 1000, dtype=np.int64)
    distance[start] = 0
    
    # Initialize set of unvisited nodes
    unvisited = set(range(num_nodes))
    
    # Initialize previous node array to track the path
    previous = np.full(num_nodes, None)
    
    while unvisited:
        # Find the node with the minimum distance
        min_node = min(unvisited, key=lambda node: distance[node])
        
        # Remove the minimum node from unvisited set
        unvisited.remove(min_node)
        
        # Update distances for neighboring nodes
        for neighbor in range(num_nodes):
            if adj_matrix[min_node][neighbor] == 1:
                new_distance = distance[min_node] + 1
                if new_distance < distance[neighbor]:
                    distance[neighbor] = new_distance
                    previous[neighbor] = min_node
    
    # Build the shortest path
    path = deque()
    current = end
    while current is not None:
        path.appendleft(current)
        current = previous[current]
    
    return list(path)

In [290]:
adj_matrix = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 1, 0, 0],
    [1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 1, 1],
    [0, 0, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 0]
])

start_node = 355
end_node = 166

shortest_path_nodes = shortest_path(driver_network, start_node, end_node)

print("Shortest path between nodes", start_node, "and", end_node, ":")
print(shortest_path_nodes)

Shortest path between nodes 355 and 166 :
[355, 395, 264, 297, 373, 178, 155, 166]


In [291]:
def lap_time_to_seconds(lap_time_str):
    try:
        if pd.isnull(lap_time_str):
            return np.inf
        minutes, seconds = map(float, lap_time_str.split(':'))
        return int(minutes) * 60 + float(seconds)
    except (ValueError, AttributeError):
        return np.inf

cleaning the drivers name from qualifying dataset 1. 

clean the drivers name for all special characters and spaces

In [292]:
new = quali_results_I['Car'].str.split(" ",expand=True)
quali_results_I['DriverName'] = quali_results_I['Driver'].str[:-4].str.replace(" ","").str.replace("-","").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
quali_results_I['DriverName'] = np.where((quali_results_I['DriverName'].str.contains('JacquesVilleneuve')) & (quali_results_I['season'] == 1983),quali_results_I['DriverName'].str.replace("JacquesVilleneuve","JacquesVilleneuveSr."),quali_results_I['DriverName'])
quali_results_I['Constructor'] = quali_results_I['Car'].str.split(" ",expand=True)[0]

In [293]:
driverInfo['DriverName'] = driverInfo['givenName'].str.replace(" ","").str.replace("-","").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')+driverInfo['familyName'].str.replace(" ","").str.replace("-","").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
join_df = pd.merge(quali_results_I,driverInfo,on=['season','DriverName'],how='left')[['Pos','Driver','Car','Time','season','round','DriverName','driverId','constructorId','Constructor']]
final_df = pd.merge(join_df,constructorInfo[['constructorId','name','season']],on=['season','constructorId'],how='left')
final_df['name'] = final_df['name'].astype(str)
final_df['Constructor'] = final_df['Constructor'].astype(str)
def check_partial_match(row):
    return ((row['Constructor'] in row['name']) |
            (('Larrousse' in row['name']) & ('Lola Ford' in row['Car']) & any(row['season'] == value for value in (1987,1988))) |
            (('Larrousse' in row['name']) & ('Lola Lamborghini' in row['Car']) & any(row['season'] == value for value in (1989,1990))) |
            (('Larrousse' in row['name']) & ('Venturi Lamborghini' in row['Car']) & (row['season'] == 1992))
           )

final_df['ConstructorMatch'] = final_df.apply(check_partial_match,axis=1)
true_df = final_df[final_df['ConstructorMatch'] == True].rename(columns={'Pos':'position'})
true_df['Q_best'] = true_df['Time'].apply(lap_time_to_seconds)

In [294]:
true_df[(true_df['driverId'] == 'michael_schumacher') & (true_df['position'] == '1')].shape

(50, 13)

In [295]:
quali_results_II['Q1_sec'] = quali_results_II['Q1'].apply(lap_time_to_seconds)
quali_results_II['Q2_sec'] = quali_results_II['Q2'].apply(lap_time_to_seconds)
quali_results_II['Q3_sec'] = quali_results_II['Q3'].apply(lap_time_to_seconds)
quali_results_II['Q_best'] = quali_results_II[['Q1_sec', 'Q2_sec','Q3_sec']].min(axis=1)

In [296]:
interested_df_II = quali_results_II[['season','round','position','driverId','constructorId','Q_best']]
interested_df_I = true_df[['season','round','position','driverId','constructorId','Q_best']]
interested_df = pd.concat([interested_df_I,interested_df_II],ignore_index=True)

In [297]:
join_df = pd.merge(interested_df,interested_df,how="left",on=['season','round','constructorId'])
join_df = join_df.loc[join_df['driverId_x'] != join_df['driverId_y']]
join_df['Q_diff'] = join_df['Q_best_x'] - join_df['Q_best_y']
join_df['Diff_Percent'] = np.maximum(join_df['Q_best_x'],join_df['Q_best_y'])/np.minimum(join_df['Q_best_x'],join_df['Q_best_y'])

In [438]:
quali_agg = join_df[(~np.isinf(join_df['Q_diff'])) & (join_df['Diff_Percent'] < 1.07)] \
.groupby(['driverId_x','driverId_y','constructorId']) \
.agg(Q_diff_mean=('Q_diff', 'mean'),count=('Q_diff', 'count')).reset_index()

#quali_agg = quali_agg.loc[(quali_agg['count'] > 3) & (quali_agg['constructorId'] == 'mclaren')]
quali_agg = quali_agg.loc[(quali_agg['count'] > 3)]

In [439]:
drivers = quali_agg.groupby(['driverId_x'])['driverId_y'].apply(list) \
.reset_index(name='teammates').rename(columns={'driverId_x':'driverId'})
poles = interested_df.groupby(['driverId']).agg \
(poles = ('position', lambda x: ((x == '1') | (x == 1)).sum()),
count = ('round','count')).reset_index()
poles['pole_pct'] = poles['poles']/poles['count']
drivers = pd.merge(drivers, poles, how="left",on=['driverId'])

In [440]:
number_drivers = drivers.shape[0]
driver_network = np.zeros((number_drivers,number_drivers),dtype=float)
for i in range(0,number_drivers):
    for j in drivers['teammates'][i]:
        ind = drivers.loc[drivers['driverId'] == j].index
        time_diff = quali_agg.loc[(quali_agg['driverId_x'] == drivers['driverId'][i]) & \
                                               (quali_agg['driverId_y'] == j)]['Q_diff_mean'].iloc[0]
        if time_diff > 0:
            driver_network[i][ind] = time_diff

In [441]:
a = np.all(driver_network == 0, axis=1).astype(int).reshape(-1, 1)
e = np.ones((number_drivers, 1))
#v = np.ones((number_drivers, 1)) / number_drivers
poles = (drivers['poles']/np.sum(drivers['pole_pct'])).values.reshape(-1,1)
v = poles

'''
# Check for dangling nodes
dangling_nodes = np.all(driver_network == 0, axis=1)

# Handle dangling nodes
if np.any(dangling_nodes):
    # Calculate the sum of PageRank mass from dangling nodes
    dangling_mass = v[dangling_nodes].sum()

    # Redistribute the mass uniformly to all nodes
    v[dangling_nodes] = dangling_mass / len(v)
'''


ae = np.dot(a, e.T) / number_drivers
non_zero_rows = ~np.all(driver_network == 0, axis=1)
H = np.zeros_like(driver_network, dtype=float)
H[non_zero_rows] = driver_network[non_zero_rows] / driver_network[non_zero_rows].sum(axis=1, keepdims=True)
H = H + ae

In [442]:
alpha = 0.85
G = alpha * H + (1 - alpha) * v
#G = alpha * H + (1 - alpha) * e * v.T
pagerank_vector = np.ones((G.shape[0], 1)) / G.shape[0]

# Set the convergence threshold
epsilon = 1e-8

# Set the maximum number of iterations (adjust as needed)
max_iterations = 10000

# Power iteration
for iteration in range(max_iterations):
    # Update the PageRank vector
    new_pagerank_vector = np.dot(G, pagerank_vector)

    # Check for convergence
    #print(np.dot(pagerank_vector.T,new_pagerank_vector)/(np.linalg.norm(pagerank_vector)*np.linalg.norm(new_pagerank_vector)))
    #print(np.linalg.norm(new_pagerank_vector - pagerank_vector, ord=1))
    if np.linalg.norm(new_pagerank_vector - pagerank_vector, ord=1) < epsilon:
        break

    # Normalize the PageRank vector
    pagerank_vector = new_pagerank_vector / np.sum(new_pagerank_vector)

In [443]:
from collections import defaultdict
driver_dict = defaultdict()
for i in range(0,len(pagerank_vector)):
    driver_dict[drivers.iloc[i]['driverId']] = pagerank_vector[i][0]

In [444]:
sorted(driver_dict.items(), key=lambda item: item[1])

[('gounon', 4.138152483622514e-11),
 ('brabham', 1.5587867241434856e-09),
 ('larrauri', 1.5587867241434856e-09),
 ('garde', 3.224019683922281e-08),
 ('modena', 5.8717412202242776e-08),
 ('doornbos', 1.0806871066414977e-07),
 ('friesacher', 1.0806871066414977e-07),
 ('monteiro', 1.5131490175290796e-07),
 ('chandhok', 2.1401751842803333e-07),
 ('poele', 4.5108073338461045e-07),
 ('lavaggi', 5.006427810565004e-07),
 ('weidler', 6.566552881394916e-07),
 ('gene', 8.687215459143192e-07),
 ('montermini', 8.687215459143192e-07),
 ('mazepin', 9.25473044535782e-07),
 ('beretta', 9.58962373847646e-07),
 ('inoue', 1.0174354035788225e-06),
 ('dalmas', 1.0329330282577454e-06),
 ('tuero', 1.1992870169307842e-06),
 ('ambrosio', 1.2144451180967557e-06),
 ('grassi', 1.2144451180967557e-06),
 ('pic', 1.2144451180967557e-06),
 ('belmondo', 1.327396623426917e-06),
 ('fittipaldi', 1.459615438493524e-06),
 ('mcnish', 1.6614442188057833e-06),
 ('de_vries', 1.8245720118331792e-06),
 ('lawson', 1.82457201183317

In [301]:
import networkx as nx
graph = nx.Graph(driver_network)

In [354]:
list(nx.connected_components(graph))

[{0,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  59,
  60,
  61,
  62,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  85,
  86,
  87,
  88,
  89,
  91,
  92,
  93,
  95,
  96,
  97,
  98,
  100,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,

In [352]:
drivers[drivers['driverId'] == 'max_verstappen']

,driverId,teammates,poles,count,pole_pct
132,max_verstappen,"[albon, gasly, perez, ricciardo, sainz]",33,185,0.178378


In [226]:
H

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.

In [227]:
v

array([[0.        ],
       [0.00909091],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.025     ],
       [0.00227273],
       [0.00227273],
       [0.04318182],
       [0.00909091]])

In [228]:
poles

array([[0.   ],
       [0.1  ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.275],
       [0.025],
       [0.025],
       [0.475],
       [0.1  ]])

In [230]:
v.sum()

0.09090909090909091

In [231]:
poles.sum()

1.0